# Python Numeric Data Analysis - Pandas

* For the times when you have Structured data, CSV, spreadsheets, R dataframes, SQL tables.

* Like numpy tables but more sophisticated labelling of rows and columns.  Good at dealing with missing and messy data. Heterogeneous data types. Time series data.

* Clean up and explore data, prepare it for analysis.

* Analyse or pass on to other systems (Scikit-learn, tensorflow, etc)

In [ ]:
# The usual suspects
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
%matplotlib inline
plt.rcParams['figure.figsize'] = [14, 8]

# And pandas
import pandas as pd


## Understanding the Pandas classes

### Series

Like a numpy array but with index labels:

In [ ]:
s = pd.Series([1.1, 2.2, 3.3, 4.4])
s

In [ ]:
s[1:3]

In [ ]:
s = pd.Series([1.1, 2.2, 3.3, 4.4], index=["alice", "bob", "charles", "diana"])
s

In [ ]:
s['charles']

In [ ]:
s['alice':'charles':2]

In [ ]:
s.index

In [ ]:
s.values

In [ ]:
populations = pd.Series({
    "London":8173941,
    "Birmingham":1085810,
    "Glasgow": 590507,
    "Liverpool": 552267, 
    "Bristol": 535907
})
populations

A lot like a Python dict...

In [ ]:
for k in populations.keys():
    print(k)

but with ordering, and the power of numpy arrays:

In [ ]:
populations / 1000000

In [ ]:
populations.std()

In [ ]:
populations.idxmax()

In [ ]:
populations[populations > 1000000]

c.f.
`SELECT * FROM populations WHERE value > 1000000;`

Percentage of men in the population:

In [ ]:
male_percent = pd.Series({
    "London": 49.12,
    "Birmingham": 49.42,
    "Leeds": 49.43,
    "Glasgow": 47.73,
    "Bristol": 49.59,
})
male_percent

Percentage of women:

In [ ]:
100 - male_percent

In [ ]:
female_pops = populations * (100 - male_percent) / 100.0
female_pops

In [ ]:
female_pops[ female_pops.notnull() ].astype(int)

All pretty clever, but that's just the Series.  There's a more powerful class...

---

### DataFrames

![Dataframe](dataframe.png)

## A quick look at a dataframe

In [ ]:
df = pd.read_excel("landmarks.xls", sheet_name="landmarks")
df

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.index

### Columns come first

It's *important* to appreciate that the first axis in a dataframe selects *columns*, not *rows*.

In [ ]:
df['confidence']

In [ ]:
type(df['confidence'])

In [ ]:
df['confidence'].values.mean()

In [ ]:
df['confidence'].mean()

In [ ]:
df['confidence'][23:26]

Note that this is still a Series, so you need to respect its indexing:

In [ ]:
df['confidence'][23:26][23]

## A potential source of confusion

So the dataframe is indexed first by the columns:

    df['confidence']

In [ ]:
for i in df.keys():
    print(i)

If you're used to Numpy arrays, note that you can't normally do:

    df[0]
    
unless there happens to be a *column* named '0'.

If you do want to get a *row* by number, there's an attribute for that:

In [ ]:
df.iloc[2]  # returns a Series for the row

In [ ]:
df.iloc[2]['y_0']

`iloc` can take other selectors, such as a slice, which will return a dataframe:

In [ ]:
df.iloc[-20::2]

**IMPORTANT ALERT!**

Now, because slicing a few rows is a sufficiently common operation, as a shortcut you *can* index into a dataframe **using a slice** and it will return rows.

    df[10:20]

is basically equivalent to

    df.iloc[10:20]

Why am I emphasising this?  Because it's different from NumPy arrays and it confused me initially:

* You can use `df[:10]` to refer to the first 10 rows, but you can't use `df[0]` to refer to the first row.
* However, `df[0]` will sometimes work, and it will give you a column!

In [ ]:
for p in df[:5]:
    print(p)

OK, are we happy?

## Back to the columns

If the column name is suitable, you can refer to it as an attribute:

In [ ]:
df.confidence.max()

In [ ]:
df[ ['frame', 'timestamp', 'confidence'] ].head()

In [ ]:
df.describe()

# Some sample data - the Lab Weather Station

The lab has a weather station at https://www.cl.cam.ac.uk/research/dtg/weather/.

![dials](https://www.cl.cam.ac.uk/research/dtg/weather/images/current-dials.png?)

The data is collated into various downloadable files:

In [ ]:
df = pd.read_csv('https://www.cl.cam.ac.uk/research/dtg/weather/weather-raw.csv', header=None)
df

Note that if the columns don't have names as headers, they'll be given numbers:

In [ ]:
df[0]

Let's give the columns names:

In [ ]:
df = pd.read_csv(
    'https://www.cl.cam.ac.uk/research/dtg/weather/weather-raw.csv', 
    names=[
        'timestamp', 'temp_dc','humidity', 'dewpoint_dc', 'pressure_mbar', 
        'mean_wind_speed_dk', 'ave_wind_bearing', 'sunshine_ch', 'rainfall_um', 'max_wind_dk'
    ],
    parse_dates=['timestamp'])

df.head(10)

In [ ]:
df.dtypes

In [ ]:
df.timestamp.head()

In [ ]:
df.temp_dc[:10000].plot();
# Or you can do plt.plot(df.temp_dc[:10000])

You can easily create new columns:

In [ ]:
df['temp']            = df['temp_dc'] / 10.0
df['dewpoint']        = df['dewpoint_dc'] / 10.0
df['mean_wind_kts']   = df['mean_wind_speed_dk'] / 10.0
df['max_wind_kts']    = df['max_wind_dk'] / 10.0
df['sunshine_hours']  = df['sunshine_ch'] / 100.0
df['rainfall_mm']     = df['rainfall_um'] / 1000.0
df.tail()

For us, the time is more useful as an index:

In [ ]:
df.set_index('timestamp', inplace=True)
df

In [ ]:
df.index

Note 'inplace'. Some methods have names like 'set_index' but don't change the original by default.

In [ ]:
df['humidity']['2018-11-08 21:30:00']

In [ ]:
df['humidity'][datetime(year=2018, month=11, day=8, hour=21, minute=30, second=0)]

In [ ]:
df.temp.plot();

In [ ]:
df[["temp", "rainfall_mm"]].plot();

Henceforth, let's ignore the data after Sept 2015.

In [ ]:
df = df[ :datetime(year=2015, month=9, day=30) ]
df.temp.plot();

In [ ]:
print(df.temp.max(), df.temp.idxmax())
print(df.temp.min(), df.temp.idxmin())

In [ ]:
df['temp'].quantile(0.99)

In [ ]:
df.temp.idxmin() - df.temp.idxmax()

In [ ]:
df.info()

In [ ]:
df.plot(kind='scatter', x='mean_wind_kts', y='temp');

In [ ]:
df['temp'][df.index.hour == 0][5000:7000].plot()
df['temp'][df.index.hour == 14][5000:7000].plot();

# GroupBy

In [ ]:
df.index.month

In [ ]:
monthgrouper = df.groupby(df.index.month)
monthgrouper

Not very helpful - what's happening behind the scenes?

In [ ]:
monthgrouper.groups

In [ ]:
for month, groupframe in monthgrouper:
    groupframe['temp'].plot(style='.', label=month)
plt.legend();

Maybe plot the first 10000...

In [ ]:
monthgrouper.mean()

In [ ]:
for month, groupframe in monthgrouper:
    print("month:",month)
    groupframe.plot(x='temp', y='mean_wind_kts', kind='scatter', xlim=[-10,35], ylim=[0, 50])
    plt.show()

# That's all for now!

Suggest you Google for '[A gallery of interesting Jupyter Notebooks](https://github.com/jupyter/jupyter/wiki/A-gallery-of-interesting-Jupyter-Notebooks)'